In [45]:
# imports
import json, re
from pprint import pprint
from lxml import etree
from pattern.nl import parse, split, parsetree
from collections import defaultdict, Counter
import unicodedata, unidecode, codecs
from nltk.tokenize import sent_tokenize, word_tokenize


In [3]:
# files
#MWE
#topics = json.load(codecs.open('D:\\4. Data\\Amazones_Forum_Export_JSON\\MWE_topic.json'))
#posts = json.load(codecs.open('D:\\4. Data\\Amazones_Forum_Export_JSON\\MWE.json'))
#regular
forums = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-35-45 _amazones_forums_export.json'))
topics = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-36-51_amazones_forum_topics_export.json'))
posts = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-39-20_amazones_forum_posts_export.json'))
users = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-39-20_amazones_users_export.json'))

In [6]:
# remove non-ascii characters from file (otherwise they will become part of the tokens)
def remove_non_ascii(text):
    return unidecode.unidecode(text)
    return ''.join([i if ord(i) < 128 else ' ' for i in text])

In [5]:
def cleanup(text):
    #remove all links, images, quotes, and emailaddresses
    text=re.sub('<a.*?>(.*?)</a>','',text) #remove links
    text=re.sub('(http:|www)\S*','',text) #remove links without markup
    text=re.sub('\[\\\/url\]','',text)
    text=re.sub('<img.*?/>', '',text) #remove images
    text=re.sub('<div class="bb-quote">((\s|\S)*?)</div>','',text) #remove quotes
    text=re.sub('<script.*?>([\S\s]*?)</script>','',text) #remove emailaddresses

    #replace all emoticon-icons
    text=re.sub('<img.*?title="(.*?)".*?/>', '(EMO:\\1)',text) #replace emoticons by textual indicators 

    # replace (most) sideways latin emoticons
    text=re.sub('[^>]:-?(\)|\])','(EMO:smiley)',text)
    text=re.sub(u'☺️','(EMO:smiley)',text)
    text=re.sub('[^>]:-?(\(|\[)','(EMO:sad)',text)
    text=re.sub(';-?(\)|\])','(EMO:wink)',text)
    text=re.sub(r'(:|;|x|X)-?(D)+\b','(EMO:laugh)',text)
    text=re.sub(':-?(/|\\\|\|)','(EMO:frown)',text)
    text=re.sub(r'(:|;)-?(p|P)+\b','(EMO:cheeky)',text)
    text=re.sub('(:|;)(\'|\")-?(\(|\[)','(EMO:cry)',text)
    text=re.sub('\<3+','(EMO:heart)',text)
    text=re.sub(u'❤️','(EMO:heart)',text)
    text=re.sub('((\>:-?(\(|\]))|(\>?:-?@))','(EMO:angry)',text)
    text=re.sub('\>:-?(\)|\])','(EMO:evil)',text)
    text=re.sub(r'(:|;)-?(O|o|0)+\b','(EMO:shock)',text)
    text=re.sub('(:|;)-?(K|k|x|X)','(EMO:kiss)',text)
    # :s
    # :x is eigenlijk geen kus, geloof ik...


    #other important adjustments:
    text=re.sub('m\'?n\s','mijn ',text) # replacing m'n and mn with mijn, so it gets parsed correctly.
    text=re.sub('z\'?n\s','zijn ',text) # replacing z'n and zn with zijn
    text=re.sub('d\'?r\s','haar ',text) # replacing d'r and dr with haar (only if followed by space, so dr. stays dr.)

    # replace all emoticons (and other things) written between double colons
    text=re.sub(':([a-zA-Z]+):','(EMO:\\1)',text)

    # remove remaining markup
    text=re.sub('</?(ol|style|b|p|em|u|i|strong|br|span|div|blockquote|li)(.*?)/?>','',text)
    text=re.sub('(\[|\]|\{|\})', '',text)

    # separate text from punctuation (may cause double/triple spaces - does not matter at this point)
    text=re.sub('(\.{2,}|/|\)|,|!|\?)','\\1 ',text) # space behind
    text=re.sub('(/|\()',' \\1',text) # space in front
    text=re.sub('(\w{2,})(\.|,)','\\1 \\2 ',text) #space 'between'

    return(text)

In [59]:
search_for = ['borst', 'borstkanker', 'operatie', 'ziekenhuis', 'vrouwen', 'arts', 'oncoloog', 'werk', 'topic', 'verhaal', 'behandeling', 'uitslag', 'chirurg', 'bestraling', 'hoofd', 'huisarts', 'man', 'reconstructie', 'amputatie']

context=defaultdict(list)

for x in topics:
    if len(search_for)==0:
        print 'break 3a'
        break
    else:
        s = parse(remove_non_ascii(cleanup(x["Body"])),encoding="utf-8").split()
        for sentence in s:
            if len(search_for)==0:
                print 'break 2a'
                break
            else:
                for word in sentence:
                    if word[0] in search_for:
                        context[word[0]].append(sentence)
                        print(word[0],len(context[word[0]]))
                        if len(context[word[0]])>=15:
                            search_for.remove(word[0])
                            print len(search_for)
                        if len(search_for)==0:
                            print 'break 1a'
                            break

            for y in reversed(posts):
                if len(search_for)==0:
                    print 'break 3b'
                    break
                else:
                    if y['Forum Topic ID'] == x['Nid']:
                        s = parse(remove_non_ascii(cleanup(y["Body"])),encoding="utf-8").split()
                        for sentence in s:
                            if len(search_for)==0:
                                print 'break 2b'
                                break
                            else:
                                for word in sentence:
                                    if word[0] in search_for:
                                        context[word[0]].append(sentence)
                                        print(word[0],len(context[word[0]]))
                                        if len(context[word[0]])>=15:
                                            search_for.remove(word[0])
                                            print len(search_for)
                                        if len(search_for)==0:
                                            print 'break 1b'
                                            break

(u'ziekenhuis', 1)
(u'vrouwen', 1)
(u'borst', 1)
(u'behandeling', 1)
(u'behandeling', 2)
(u'chirurg', 1)
(u'chirurg', 2)
(u'chirurg', 3)
(u'behandeling', 3)
(u'behandeling', 4)
(u'chirurg', 4)
(u'chirurg', 5)
(u'chirurg', 6)
(u'operatie', 1)
(u'behandeling', 5)
(u'behandeling', 6)
(u'chirurg', 7)
(u'chirurg', 8)
(u'chirurg', 9)
(u'amputatie', 1)
(u'behandeling', 7)
(u'behandeling', 8)
(u'chirurg', 10)
(u'chirurg', 11)
(u'chirurg', 12)
(u'behandeling', 9)
(u'behandeling', 10)
(u'chirurg', 13)
(u'chirurg', 14)
(u'chirurg', 15)
18
(u'reconstructie', 1)
(u'behandeling', 11)
(u'behandeling', 12)
(u'behandeling', 13)
(u'behandeling', 14)
(u'behandeling', 15)
17
(u'uitslag', 1)
(u'werk', 1)
(u'operatie', 2)
(u'bestraling', 1)
(u'borst', 2)
(u'werk', 2)
(u'operatie', 3)
(u'bestraling', 2)
(u'borst', 3)
(u'werk', 3)
(u'operatie', 4)
(u'bestraling', 3)
(u'borst', 4)
(u'werk', 4)
(u'operatie', 5)
(u'bestraling', 4)
(u'borst', 5)
(u'werk', 5)
(u'operatie', 6)
(u'bestraling', 5)
(u'borst', 6)
(u'we

pprint(context.items())

In [99]:
matchdict = defaultdict(list)
for x in topics:
    s = parse(remove_non_ascii(cleanup(x["Body"])),encoding="utf-8").split()
    for sentence in s:
        for word in sentence:
            match = re.search(r'((\s|^)\w(\.\w+){1,}\.?)',word[0])
            if match == None:
            # alles met alleen hoofdletters (r)
                match = re.search(r'[A-Z]{2,}',word[0])
                if match == None:
                    pass
                else:
                    matchdict[match.group(0)].append('1')
            else:
                matchdict[match.group(0)].append('1')


In [100]:
count = []
for item in matchdict:
    count.append((item, len(matchdict[item])))

In [102]:
sortedcount = sorted(count, key=lambda x: x[1], reverse=True)

with open('afkortingen.txt','w') as f:
    f.write(json.dumps(sortedcount))

In [74]:
zin = "deze zin bevat. afkortingen.zoals b.v. deze, en WC en a.k.a! Maar ook d.f.f.he.e.f.v. bijvoorbeeld en AIDS"
for word in word_tokenize(zin):
    # alles met punten erin
    match = re.search(r'((\s|^)\w(\.\w+){1,}\.?)',word)
    if match == None:
        # alles met alleen hoofdletters (r)
        match = re.search(r'[A-Z]{2,}',word)
        if match == None:
            pass
        else:
            print match.group(0)
    else:
        print match.group(0)



b.v.
WC
a.k.a
d.f.f.he.e.f.v
AIDS
